In [3]:
import pandas as pd
import numpy as np

np.__version__

'1.19.5'

# Pull in data (prepped in other notebook)

In [4]:
lyric_lines_ready_for_analysis = pd.read_csv('./data/3 OUTPUT One lyric lines per row, cleaned.csv')

In [5]:
lyric_lines_ready_for_analysis

,performer,song,generic_genre,lyric_line,chart_debut,chart_debut_date_as_a_decimal
0,John Lennon,#9 Dream,ROCK,So long ago,1974-12-21,1974.97
1,John Lennon,#9 Dream,ROCK,"Was it in a dream, was it just a dream?",1974-12-21,1974.97
2,John Lennon,#9 Dream,ROCK,"I know, yes I know",1974-12-21,1974.97
3,John Lennon,#9 Dream,ROCK,"Seemed so very real, it seemed so real to me",1974-12-21,1974.97
4,John Lennon,#9 Dream,ROCK,Took a walk down the street,1974-12-21,1974.97
...,...,...,...,...,...,...
192476,J. Cole & Lil Baby,pride.is.the.devil,HIP_HOP,"I'm addicted to promethazine, it's crazy, yeah...",2021-05-29,2021.41
192477,J. Cole & Lil Baby,pride.is.the.devil,HIP_HOP,"All this money coming in, it drive me crazy no...",2021-05-29,2021.41
192478,J. Cole & Lil Baby,pride.is.the.devil,HIP_HOP,I'll be crazy if I blow it,2021-05-29,2021.41
192479,J. Cole & Lil Baby,pride.is.the.devil,HIP_HOP,And it left so many R.I.P.,2021-05-29,2021.41


# EMBED WORDS USING Universal Encoder

In [6]:
#@title Load the Universal Sentence Encoder's TF Hub module
from absl import logging

import tensorflow as tf

import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

In [7]:
model = hub.load('./data/universal-sentence-encoder-large_5/')

2022-12-08 13:50:51.519493: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
def embed(input):
    return model(input)

## Ok, so the notebook crashes trying to embed *all at once*, so let's break this up:

In [16]:

lyric_lines_ready_for_analysis.iloc[0:1]

,performer,song,generic_genre,lyric_line,chart_debut,chart_debut_date_as_a_decimal
0,John Lennon,#9 Dream,ROCK,So long ago,1974-12-21,1974.97


In [ ]:
# %timeit embed(lyric_lines_ready_for_analysis.head(1000)['lyric_line'])

In [ ]:
# message_embeddings = embed(lyric_lines_ready_for_analysis['lyric_line'])

# Dimension reduction

## Dimension reduction with TSNE

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
# tsne_transformed_embeddings = TSNE(
#     n_components=2,
#     learning_rate='auto',
#     init='pca'
# ).fit_transform(message_embeddings)

In [ ]:
def concat_x_and_y(x_and_y, df):
    return pd.concat(
    [df, pd.DataFrame(x_and_y, columns=['x', 'y'])],
    axis=1
)

In [ ]:
# tsne_lyrics_and_artist = concat_x_and_y(
#     tsne_transformed_embeddings, 
#     lyric_lines_ready_for_analysis
# )

In [ ]:
# sns.scatterplot(data=tsne_lyrics_and_artist, x='x', y='y')

# Dimension reduction with UMAP

In [ ]:
import umap
from sklearn.preprocessing import StandardScaler

In [ ]:
reducer = umap.UMAP()

In [ ]:
embedding = reducer.fit_transform(
    StandardScaler().fit_transform(message_embeddings)
)

In [ ]:
umap_x_and_y_added = concat_x_and_y(
    embedding, 
    df=lyric_lines_ready_for_analysis
)
umap_x_and_y_added

In [ ]:
sns.scatterplot(data=umap_x_and_y_added, x='x', y='y')

## Export to CSV

In [ ]:
def normalize_zero_to_one(s):
    return (s - s.min()) / (s.max() - s.min())

def export_normalized_data(data):
    temp = data.copy().dropna()
    temp['x'] = normalize_zero_to_one(temp.x)
    temp['y'] = normalize_zero_to_one(temp.y)
    temp['x'] = temp['x'].round(3)
    temp['y'] = temp['y'].round(3)
    export = temp
    export.to_csv('./data/viz ready export 1976 - 1980.csv', index=False)
    return export

In [ ]:
the_export = export_normalized_data(umap_x_and_y_added)
the_export

# List of all: genres, artists

In [ ]:
my_set = set()
the_export['Performer'].unique()

In [ ]:
from ast import literal_eval

my_set = set()
the_export['spotify_genre'].apply(
    lambda val: val and my_set.update(literal_eval(val))
)
my_set

In [ ]:
rap_genres = [genre for genre in list(my_set) if 'rap' in genre]

In [ ]:
pop_genres = [genre for genre in list(my_set) if 'pop' in genre]
pop_genres